In [1]:
import pandas as pd

In [2]:
import requests

url = 'https://annamalaiuniversity.ac.in/adm/coun_admlist_be.php?admfor=910955a907e739b81ec8855763108a29'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text)
dfs=dfs[0]
dfs.columns=["SI.NO","Appl NO","Name","Branch","Dob","Community","Payment"]
display(dfs)

,SI.NO,Appl NO,Name,Branch,Dob,Community,Payment
0,ONLINE ADMISSION -,NaN,NaN,NaN,NaN,NaN,NaN
1,SPL The last date for payment of Admission Fee...,NaN,NaN,NaN,NaN,NaN,NaN
2,Sl.No.,Appl.No.,Name,Branch,DOB,Community,Payment
3,1,201782,MOHAN K,Computer Science and Engineering (Artificial I...,17/08/2004,BC,NaN
4,Sl.No.,Appl.No.,Name,Branch,DOB,Community,Payment
...,...,...,...,...,...,...,...
759,2,610065,PADMAPRIYA S,ELECTRONICS AND INSTRUMENTATION ENGINEERING,NaN,BC,NaN
760,Sl.No.,Appl.No.,Name,Branch,DOB,Community,Payment
761,1,605532,DHANUSH KUMAR M,ELECTRICAL AND ELECTRONICS ENGINEERING,NaN,SC,NaN
762,2,607692,AKASH V,ELECTRICAL AND ELECTRONICS ENGINEERING,NaN,SC,NaN


In [3]:
dfs.drop([0,1],inplace=True)
display(dfs)

,SI.NO,Appl NO,Name,Branch,Dob,Community,Payment
2,Sl.No.,Appl.No.,Name,Branch,DOB,Community,Payment
3,1,201782,MOHAN K,Computer Science and Engineering (Artificial I...,17/08/2004,BC,NaN
4,Sl.No.,Appl.No.,Name,Branch,DOB,Community,Payment
5,1,404123,DISHYA S,COMPUTER SCIENCE AND ENGINEERING,NaN,BC,NaN
6,2,337886,KALAI SELVAN M,COMPUTER SCIENCE AND ENGINEERING,NaN,BC,NaN
...,...,...,...,...,...,...,...
759,2,610065,PADMAPRIYA S,ELECTRONICS AND INSTRUMENTATION ENGINEERING,NaN,BC,NaN
760,Sl.No.,Appl.No.,Name,Branch,DOB,Community,Payment
761,1,605532,DHANUSH KUMAR M,ELECTRICAL AND ELECTRONICS ENGINEERING,NaN,SC,NaN
762,2,607692,AKASH V,ELECTRICAL AND ELECTRONICS ENGINEERING,NaN,SC,NaN


In [4]:
all_std=dfs[dfs["Name"]!="Name"]
all_std.to_excel(r"all_std.xlsx",index=False)
display(all_std)

,SI.NO,Appl NO,Name,Branch,Dob,Community,Payment
3,1,201782,MOHAN K,Computer Science and Engineering (Artificial I...,17/08/2004,BC,NaN
5,1,404123,DISHYA S,COMPUTER SCIENCE AND ENGINEERING,NaN,BC,NaN
6,2,337886,KALAI SELVAN M,COMPUTER SCIENCE AND ENGINEERING,NaN,BC,NaN
7,3,400218,YOGESWARI MEYYAMMAI M,COMPUTER SCIENCE AND ENGINEERING,NaN,OC,NaN
8,4,318082,Niranjana. T,COMPUTER SCIENCE AND ENGINEERING,NaN,BC,NaN
...,...,...,...,...,...,...,...
758,1,600414,PHIZAL I,ELECTRICAL AND ELECTRONICS ENGINEERING,NaN,BC(Muslim),NaN
759,2,610065,PADMAPRIYA S,ELECTRONICS AND INSTRUMENTATION ENGINEERING,NaN,BC,NaN
761,1,605532,DHANUSH KUMAR M,ELECTRICAL AND ELECTRONICS ENGINEERING,NaN,SC,NaN
762,2,607692,AKASH V,ELECTRICAL AND ELECTRONICS ENGINEERING,NaN,SC,NaN


In [5]:
cse=dfs[dfs["Branch"]=="COMPUTER SCIENCE AND ENGINEERING"]
display(cse)

,SI.NO,Appl NO,Name,Branch,Dob,Community,Payment
5,1,404123,DISHYA S,COMPUTER SCIENCE AND ENGINEERING,NaN,BC,NaN
6,2,337886,KALAI SELVAN M,COMPUTER SCIENCE AND ENGINEERING,NaN,BC,NaN
7,3,400218,YOGESWARI MEYYAMMAI M,COMPUTER SCIENCE AND ENGINEERING,NaN,OC,NaN
8,4,318082,Niranjana. T,COMPUTER SCIENCE AND ENGINEERING,NaN,BC,NaN
9,5,243318,EASTER RAJ A,COMPUTER SCIENCE AND ENGINEERING,NaN,BC,NaN
...,...,...,...,...,...,...,...
413,18,261074,GOKUL R,COMPUTER SCIENCE AND ENGINEERING,NaN,SC,NaN
414,19,372921,VIDHYA K,COMPUTER SCIENCE AND ENGINEERING,NaN,BC,NaN
415,20,282653,SEHWAG E,COMPUTER SCIENCE AND ENGINEERING,NaN,MBC/DNC,NaN
525,62,210038,BHARATH G,COMPUTER SCIENCE AND ENGINEERING,NaN,SCA,NaN


In [6]:
it=dfs[dfs["Branch"]=="INFORMATION TECHNOLOGY"]
it.to_excel("it.xlsx")
display(it)

,SI.NO,Appl NO,Name,Branch,Dob,Community,Payment
14,10,281880,Ramena Afra Banu.A,INFORMATION TECHNOLOGY,NaN,BC(Muslim),NaN
23,5,298218,SATHYA M,INFORMATION TECHNOLOGY,18/03/2005,BC,NaN
24,6,312018,SACHIN KUMAR S,INFORMATION TECHNOLOGY,NaN,SC,NaN
121,96,206394,Anjana,INFORMATION TECHNOLOGY,NaN,MBC/DNC,NaN
122,97,254059,VIGNESH V,INFORMATION TECHNOLOGY,NaN,MBC/DNC,NaN
...,...,...,...,...,...,...,...
616,153,349902,MOHAMED FAYAS M R,INFORMATION TECHNOLOGY,NaN,BC(Muslim),NaN
617,154,389278,MADHESH E,INFORMATION TECHNOLOGY,NaN,SC,NaN
724,39,313541,ABILASRI K,INFORMATION TECHNOLOGY,NaN,BC,NaN
725,40,267289,ABINESH A,INFORMATION TECHNOLOGY,NaN,SC,NaN
